In [5]:
import pandas as pd
import json
import csv
from tqdm import tqdm
import os

In [22]:
def parse_json_to_csv(input_file, output_file):
    csv_line = ['timestamp','lineID','directionID','distancefromPoint','pointID']
    c = open(output_file, 'w',newline='')
    # create the csv writer
    writer = csv.writer(c)
    # write csv header
    writer.writerow(csv_line)
    #Open json file
    with open(input_file,'r') as f:
        json_file = json.loads(f.read())
        #loop through timestamps
        i=0
        for data in json_file['data']:
            time = json_file['data'][i]['time']
            #print(time)
            #loop through all responses of a timestamp
            m=0
            for response in json_file['data'][i]['Responses']:
                #loop through all lines of a response
                j = 0
                try:
                    for lines in json_file['data'][i]['Responses'][m]['lines']:
                        lineId = json_file['data'][i]['Responses'][m]['lines'][j]['lineId']
                        #loop throgh all vehicle positions of a line
                        k=0 
                        for vehicelePosition in json_file['data'][i]['Responses'][m]['lines'][j]['vehiclePositions']:
                            directionId = json_file['data'][i]['Responses'][m]['lines'][j]['vehiclePositions'][k]['directionId']
                            distanceFromPoint = json_file['data'][i]['Responses'][m]['lines'][j]['vehiclePositions'][k]['distanceFromPoint']
                            pointId = json_file['data'][i]['Responses'][m]['lines'][j]['vehiclePositions'][k]['pointId']
                            #write to csv file
                            csv_line = [time,lineId,directionId,distanceFromPoint,pointId]
                            writer.writerow(csv_line)
                            k = k+1
                        j=j+1
                #catch null values
                except:
                    #write timestamp with null values to csv
                    csv_line = [time,'null','null','null','null']
                    writer.writerow(csv_line)
                    #print(csv_line)
                m=m+1
            i = i+1
    ##print(f"finished writing file in {output_file}")
    c.close()

In [23]:
data_folder = '/Users/alfredo.leon/Desktop/DataMiningProject/Project Data-20221021/'
docs = [data_folder + f'vehiclePosition0{i+1}' for i in range(9)]
docs.extend([data_folder + f'vehiclePosition{i+1}' for i in range(9,13)])
for doc in tqdm(docs):
    parse_json_to_csv(input_file=doc+'.json', output_file=doc+'.csv')

100%|█████████████████████████████████████| 13/13 [02:06<00:00,  9.75s/it]


### Preprocessing CSVs to make them more readable

In [46]:
def preprocess_csv(input_file):
    df = pd.read_csv(input_file, index_col=None, dtype=str)
    df['time'] = pd.to_datetime(df['timestamp'], unit='ms')
    df['year'] = df['time'].dt.year
    df['month'] = df['time'].dt.month
    df['day'] = df['time'].dt.day
    df['hour'] = df['time'].dt.hour
    df['minute'] = df['time'].dt.minute
    df['second'] = df['time'].dt.second
    df.to_csv(input_file, index=False)
##preprocess_csv('/Users/alfredo.leon/Desktop/DataMiningProject/Project Data-20221021/csvs/vehiclePosition13.csv')

In [49]:
csvs_folder = '/Users/alfredo.leon/Desktop/DataMiningProject/Project Data-20221021/csvs/'
for doc in tqdm(os.listdir(csvs_folder)):
    preprocess_csv(input_file=csvs_folder + doc)

100%|█████████████████████████████████████| 13/13 [02:05<00:00,  9.62s/it]
